In [1]:
import folium
import numpy as np
import pandas as pd
from folium.plugins import HeatMap
from folium import plugins
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.colors as mcolors

In [2]:
df_total=pd.read_csv("sorted_data_small.csv",index_col=False)
df_total.info()

FileNotFoundError: [Errno 2] No such file or directory: 'sorted_data_small.csv'

## Ashraf Attributes are 
### PRECSNO
### PRECTOTCORR
### PW small dataset


In [ ]:
df=df_total[["LAT","LON","DOY","YEAR","PW"]]
df.head(30)

,LAT,LON,DOY,YEAR,PW
0,22.5,25.5,1,2019,0.69
1,22.5,26.5,1,2019,0.75
2,22.5,27.5,1,2019,0.87
3,22.5,28.5,1,2019,0.98
4,22.5,29.5,1,2019,1.03
5,22.5,30.5,1,2019,1.07
6,22.5,31.5,1,2019,1.11
7,22.5,32.5,1,2019,1.15
8,22.5,33.5,1,2019,1.19
9,22.5,34.5,1,2019,1.23


In [ ]:

Q1 = df['PW'].quantile(0.25)
Q2 = df['PW'].quantile(0.5)
Q3 = df['PW'].quantile(0.75)
Q4 = df['PW'].quantile(0.9)
Q5 = df['PW'].quantile(0.999)
Mean=df["PW"].mean()

# Step 2: Calculate the IQR
IQR = Q3 - Q1

# Step 3: Define lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Step 4: Count outliers
outliers = df[(df['PW'] < lower_bound) | (df['PW'] > upper_bound)]
outlier_count = outliers.shape[0]

# Display outlier count
print(outlier_count)
print(Q1)
print(Q2)
print(Q3)
print(Q4)
print(Q5)
print(Mean)

1408
1.03
1.44
1.91
2.34
4.035310000000172
1.505021790511328


In [ ]:


# Create the WOY column
df['WOY'] = ((df['DOY'] - 1) // 7) + 1  # Convert DOY to Week of Year (assuming DOY starts from 1)

# Group by LAT, LON, YEAR, WOY and aggregate
weekly_df = df.groupby(['LAT', 'LON', 'YEAR', 'WOY']).agg(
    AVG_PW=('PW', 'mean'),
    MAX_PW=('PW', 'max'),
    MIN_PW=('PW', 'min')
).reset_index()

# Display the new DataFrame
weekly_df=weekly_df.sort_values(by=["YEAR",'WOY',"LAT",'LON'],ascending=[True,True,True,True])
display(len(weekly_df))
weekly_df.to_csv("Weekly_PW.csv",index=False)


C:\Users\ahmed\AppData\Local\Temp\ipykernel_25560\223000826.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['WOY'] = ((df['DOY'] - 1) // 7) + 1  # Convert DOY to Week of Year (assuming DOY starts from 1)


24115

In [ ]:

# Load Data
data = pd.read_csv("Weekly_PW.csv")
data['WOY'] = pd.to_numeric(data['WOY'], errors='coerce')
print(data.dtypes)


LAT       float64
LON       float64
YEAR        int64
WOY         int64
AVG_PW    float64
MAX_PW    float64
MIN_PW    float64
dtype: object


In [ ]:

# Function to generate and display the heatmap
def display_filtered_map(year, week, save=False):
    clear_output(wait=True)  # Clear previous map while keeping widgets visible
    display(year_selector, week_selector, button, save_button)  # Keep input cells visible

    # Filter data based on YEAR and WOY, and remove points with AVG_PW = 0
    filtered_data = data[(data['YEAR'] == year) & (data['WOY'] == week) & (data['AVG_PW'] > 0)]

    if filtered_data.empty:
        print(f"No data available for Year {year}, Week {week}")
        return

    # Create Folium map centered around Egypt
    egypt_map = folium.Map(location=[26.8206, 30.8025], zoom_start=6)

    # Normalize AVG_PW for better heatmap visualization
    min_value = filtered_data['AVG_PW'].min()
    max_value = filtered_data['AVG_PW'].max()
    
    filtered_data['Intensity'] = filtered_data['AVG_PW'].apply(
        lambda x: (x - min_value) / (max_value - min_value) if max_value > min_value else 0.5
    )

    # Prepare heatmap data with normalized intensity
    heatmap_data = filtered_data[['LAT', 'LON', 'Intensity']].values.tolist()
    
    # Add heatmap layer
    HeatMap(heatmap_data, radius=10, blur=15, max_zoom=10).add_to(egypt_map)

    # Get top 5 highest precipitation points
    top_5_points = filtered_data.nlargest(5, 'AVG_PW')

    # Define a colormap from red (high values) to blue (low values)
    cmap = mcolors.LinearSegmentedColormap.from_list("custom", ["blue", "cyan", "yellow", "orange", "red"])
    
    # Function to map intensity to color
    def intensity_to_color(intensity):
        return mcolors.to_hex(cmap(intensity))

    # Add all points with colors matching heatmap
    for _, row in filtered_data.iterrows():
        color = intensity_to_color(row['Intensity'])
        is_top5 = any(row.equals(top_5) for _, top_5 in top_5_points.iterrows()) 

        folium.CircleMarker(
            location=[row['LAT'], row['LON']],
            radius=10 if is_top5 else 5,  # Larger radius for top 5
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6 if is_top5 else 0.4,  # Make top 5 bolder
            tooltip=f"PW: {row['AVG_PW']}"
        ).add_to(egypt_map)

    # Save map only when requested
    if save:
        filename = f"PW_heatmap_{year}_W{week}.html"
        egypt_map.save(filename)
        print(f"Heatmap saved as {filename}")
    
    # Display the map
    display(egypt_map)

# Input widgets
year_selector = widgets.IntText(
    value=data['YEAR'].min(),
    description="Year:",
    min=data['YEAR'].min(),
    max=data['YEAR'].max()
)

week_selector = widgets.IntText(
    value=1,
    description="Week:",
    min=1,
    max=52
)

button = widgets.Button(description="Show Heatmap")
save_button = widgets.Button(description="Save as HTML")

# Function to update map when button is clicked
def on_button_click(b):
    display_filtered_map(year_selector.value, week_selector.value)

button.on_click(on_button_click)

# Function to save map when save button is clicked
def on_save_button_click(b):
    display_filtered_map(year_selector.value, week_selector.value, save=True)

save_button.on_click(on_save_button_click)

# Display widgets
display(year_selector, week_selector, button, save_button)

IntText(value=2019, description='Year:')

IntText(value=1, description='Week:')

Button(description='Show Heatmap', style=ButtonStyle())

Button(description='Save as HTML', style=ButtonStyle())

C:\Users\ahmed\AppData\Local\Temp\ipykernel_25560\2915794686.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Intensity'] = filtered_data['AVG_PW'].apply(


In [9]:

# Function to process each week and extract PW points
def extract_weekly_PW(data, year):
    results = []
    
    for week in range(1, data[data['YEAR'] == year]['WOY'].max() + 1):
        weekly_data = data[(data['YEAR'] == year) & (data['WOY'] == week) & (data['AVG_PW'] > 0)]
        
        if weekly_data.empty:
            print(f"Warning: No data for YEAR {year}, WOY {week}")
            results.append([week, year] + [None] * 15 + [None])
            continue
        
        top_5 = weekly_data.nlargest(min(5, len(weekly_data)), 'AVG_PW')[['LAT', 'LON', 'AVG_PW']]
        top_5['AVG_PW'] = top_5['AVG_PW'].round(3)
        avg_precip = round(weekly_data['AVG_PW'].mean(), 3)
        
        top_5_values = top_5.values.flatten().tolist()
        
        # Ensure list has 15 elements (5 points * 3 values each)
        while len(top_5_values) < 15:
            top_5_values.append(None)  # Fill missing values with None for clarity
        
        results.append([week, year] + top_5_values + [avg_precip])
    
    return results

# Process the data for all available years
all_results = []
years = data['YEAR'].unique()

for y in years:
    all_results.extend(extract_weekly_PW(data, y))

# Convert to DataFrame and save to CSV
columns = ['WOY', 'YEAR'] + [f'Top{i}_{attr}' for i in range(1, 6) for attr in ['LAT', 'LON', 'Value']] + ['Avg']
weekly_summary = pd.DataFrame(all_results, columns=columns)

# Replace None with empty strings to avoid NaNs in CSV
weekly_summary.fillna('', inplace=True)

weekly_summary.to_csv('weekly_PW_summary.csv', index=False)

print("CSV file 'weekly_PW_summary.csv' has been saved.")


CSV file 'weekly_PW_summary.csv' has been saved.
